In [43]:
from torch.nn import Linear
import json
import torch
from tokenizers.implementations import BertWordPieceTokenizer
from Bert import Bert

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

with open("../bert_impl_data/ner_class.json", "r", encoding="utf-8") as fp:
    Class = json.load(fp)

_tokenizer = BertWordPieceTokenizer("../custom/vocab.txt")
_embedding_dim = 384
_hidden_size = 3072
_num_head = 12
_out_dim = 512
max_epoch = 20
batch = 140
_num_layers = 12
vocab_size = _tokenizer.get_vocab_size()
bert = Bert(_embedding_dim, _hidden_size, _num_head, 128, _num_layers, _tokenizer)
layers = Linear(384, len(Class['id_word']), device=bert.device)
bert.load_state_dict(torch.load("../bert_impl_weights/down_stream_bert_ner.pth"))
layers.load_state_dict(torch.load("../bert_impl_weights/down_stream_bert_ner_layer.pth"))
bert.eval()

Bert(
  (tokenEmbedding): Embedding(30522, 384)
  (segmentEmbedding): Embedding(2, 384)
  (position_embedding): Embedding(128, 384)
  (encoder): ModuleList(
    (0-11): 12 x Encoder(
      (multiHeadAttention): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=384, out_features=384, bias=True)
      )
      (layerNorm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (feedForward): FeedForward(
        (linear1): Linear(in_features=384, out_features=3072, bias=True)
        (linear2): Linear(in_features=3072, out_features=384, bias=True)
        (relu): GELU(approximate='none')
      )
      (layerNorm2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (drop_out): Dropout(p=0.1, inplace=False)
    )
  )
  (linear): Linear(in_features=384, out_features=30522, bias=True)
  (softmax): Softmax(dim=-1)
  (loss_func): CrossEntropyLoss()
)

In [44]:
sentence = "He is at Tianan men and he is heading to school."
ids = _tokenizer.encode(sentence).ids
inp = torch.tensor(ids, device=device)
tokens = _tokenizer.encode(sentence).tokens[1:-1]

In [45]:
with torch.no_grad():
    score = bert.forward(inp, (inp != 0).to(device=device))
    score = layers.forward(score)
    possibility = bert.softmax(score)
    predict = torch.argmax(possibility, dim=-1)
ls = predict.tolist()[1:-1]
print(_tokenizer.decode(ids))
print(' '.join([Class['id_real'][str(i)] * len(tokens[idx].replace('##', '')) if str(i) == "10" else Class['id_real'][str(i)] for idx, i in enumerate(ls)]))

he is at tianan men and he is heading to school.
         地理政治实体前缀                                     
